In [414]:
import pandas as pd
import os
import json

In [415]:
no1 = "No.1"
no1 = no1.center(100,"=")
print(f"\n",no1)


 ================================================No.1================================================


In [416]:
# create a function to load all the JSON files inside a folder and normalize them then append the files into a data_list
def load_and_normalize(folder_path):
    data_list = []

    # loop into the folder, scanning all the files inside the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open(file_path,'r') as f:
            data = json.load(f)
            data_list.append(data)
            
    # normalize the datas inside data_list and update the data_list directly
    data_list = pd.json_normalize(data_list)
    
    # sort the datas inside data_list ny id and ts(timestamp)
    data_list = data_list.sort_values(by=['id','ts'])
    
    return data_list

In [417]:
data_folder = os.environ.get("DATA_FOLDER", "./data")
accounts_path = os.path.join(data_folder,'accounts')
cards_path = os.path.join(data_folder,'cards')
savings_accounts_path = os.path.join(data_folder,'savings_accounts')

In [418]:
# Feeling dataframe "df_accounts" with all the JSON data files located in "./data/accounts" using load_and_normalize function
df_accounts = load_and_normalize("./data/accounts")

# renaming and combining the features/columns
df_accounts["timestamp"] = df_accounts["ts"]
df_accounts["account_id"] = df_accounts["data.account_id"]
df_accounts["name"] = df_accounts["data.name"]
df_accounts["email"] = df_accounts["set.email"].combine_first(df_accounts["data.email"])
df_accounts["savings_account_id"] = df_accounts["set.savings_account_id"]
df_accounts["card_id"] = df_accounts["set.card_id"]

# combine only if there is a "set" column where it represents an update on the data with the same "name"
# combining column "set.address" with "data.address" into a new column named "address"
df_accounts["address"] = df_accounts["set.address"].combine_first(df_accounts["data.address"])

# combining column "set.phone_number" with "data.phone_number" into a new column named "phone_number"
df_accounts["phone_number"] = df_accounts["set.phone_number"].combine_first(df_accounts["data.phone_number"])

# choosing the needed columns that's been cleaned
cols = ["id","timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id"]
df_accounts = df_accounts[cols]

# grouping all the datas by using "id" as reference and feeling the empty data with the the ones above it (updating from the previous data
df_accounts = df_accounts.groupby("id").ffill()

print("\nACCOUNTS:")
print(df_accounts.to_string())


ACCOUNTS:
       timestamp account_id     name   address phone_number                    email savings_account_id card_id
0  1577863800000         a1  Anthony  New York     12345678     anthony@somebank.com                NaN     NaN
1  1577865600000         a1  Anthony  New York     87654321     anthony@somebank.com                NaN     NaN
2  1577890800000         a1  Anthony  New York     87654321     anthony@somebank.com                sa1     NaN
3  1577894400000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1     NaN
4  1577926800000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c1
5  1579078860000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1        
6  1579163400000         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c2


In [419]:
# Feeling dataframe "df_accounts" with all the JSON data files located in "./data/cards" using load_and_normalize function
df_cards = load_and_normalize("./data/cards")

# renaming and combining the features/columns
df_cards["timestamp"] = df_cards["ts"]
df_cards["card_id"] = df_cards["data.card_id"]
df_cards["card_number"] = df_cards["data.card_number"]
df_cards["monthly_limit"] = df_cards["data.monthly_limit"]

# combine only if there is a "set" column where it represents an update on the data with the same "name"
# combining column "set.credit_used" with "data.credit_used" into a new column named "credit_used"
df_cards["credit_used"] = df_cards["set.credit_used"].combine_first(df_cards["data.credit_used"])

# combining column "set.status" with "data.status" into a new column named "status"
df_cards["status"] = df_cards["set.status"].combine_first(df_cards["data.status"])

# choosing the needed columns that's been cleaned
cols = ["id","timestamp","card_id","card_number","credit_used","monthly_limit","status"]
df_cards = df_cards[cols]

# grouping all the datas by using "id" as reference and feeling the empty data with the the ones above it (updating from the previous data
df_cards = df_cards.groupby("id").ffill()

print("\nCARDS:")
print(df_cards.to_string())


CARDS:
       timestamp card_id card_number  credit_used  monthly_limit   status
0  1577926800000      c1    11112222          0.0        30000.0  PENDING
1  1578159000000      c1    11112222          0.0        30000.0   ACTIVE
2  1578313800000      c1    11112222      12000.0        30000.0   ACTIVE
3  1578420000000      c1    11112222      19000.0        30000.0   ACTIVE
4  1578654000000      c1    11112222          0.0        30000.0   ACTIVE
5  1579078800000      c1    11112222          0.0        30000.0   CLOSED
6  1579163400000      c2    12123434          0.0        70000.0  PENDING
7  1579298400000      c2    12123434          0.0        70000.0   ACTIVE
8  1579361400000      c2    12123434      37000.0        70000.0   ACTIVE


In [420]:
# Feeling dataframe "df_accounts" with all the JSON data files located in "./data/cards" using load_and_normalize function
df_saving_accounts = load_and_normalize("./data/savings_accounts")

# renaming and combining the features/columns
df_saving_accounts["timestamp"] = df_saving_accounts["ts"]
df_saving_accounts["savings_account_id"] = df_saving_accounts["data.savings_account_id"]
df_saving_accounts["status"] = df_saving_accounts["data.status"]

# combine only if there is a "set" column where it represents an update on the data with the same "name"
# combining column "set.balance" with "data.balance" into a new column named "balance"
df_saving_accounts["balance"] = df_saving_accounts["set.balance"].combine_first(df_saving_accounts["data.balance"])

# combining column "set.interest_rate_percent" with "data.interest_rate_percent" into a new column named "interest_rate_percent"
df_saving_accounts["interest_rate_percent"] = df_saving_accounts["set.interest_rate_percent"].combine_first(df_saving_accounts["data.interest_rate_percent"])

# choosing the needed columns that's been cleaned
cols = ["id","timestamp","savings_account_id","balance","interest_rate_percent","status"]
df_saving_accounts = df_saving_accounts[cols]

# grouping all the datas by using "id" as reference and feeling the empty data with the the ones above it (updating from the previous data)
df_saving_accounts = df_saving_accounts.groupby("id").ffill()

print("\nSAVING ACCOUNTS:")
print(df_saving_accounts.to_string())


SAVING ACCOUNTS:
       timestamp savings_account_id  balance  interest_rate_percent  status
0  1577890800000                sa1      0.0                    1.5  ACTIVE
1  1577955600000                sa1  15000.0                    1.5  ACTIVE
2  1578159060000                sa1  15000.0                    3.0  ACTIVE
3  1578648600000                sa1  40000.0                    3.0  ACTIVE
4  1578654000000                sa1  21000.0                    3.0  ACTIVE
5  1579078860000                sa1  21000.0                    1.5  ACTIVE
6  1579298460000                sa1  21000.0                    4.0  ACTIVE
7  1579505400000                sa1  33000.0                    4.0  ACTIVE


In [421]:
no2 = "No.2"
no2 = no2.center(100,"=")
print(f"\n",no2)


 ================================================No.2================================================


In [422]:
account_card_join_df = pd.merge(df_accounts,df_cards,on="card_id",how="outer")
account_card_join_df["timestamp"] = account_card_join_df["timestamp_y"].combine_first(account_card_join_df["timestamp_x"])
cols = ["timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id","card_number","credit_used","monthly_limit","status"]
account_card_join_df = account_card_join_df[cols].sort_values(["timestamp","card_id"])
print(f"\nMerged Table Account and Cards: \n",account_card_join_df.to_string())


Merged Table Account and Cards: 
        timestamp account_id     name   address phone_number                    email savings_account_id card_id card_number  credit_used  monthly_limit   status
0   1.577864e+12         a1  Anthony  New York     12345678     anthony@somebank.com                NaN     NaN         NaN          NaN            NaN      NaN
1   1.577866e+12         a1  Anthony  New York     87654321     anthony@somebank.com                NaN     NaN         NaN          NaN            NaN      NaN
2   1.577891e+12         a1  Anthony  New York     87654321     anthony@somebank.com                sa1     NaN         NaN          NaN            NaN      NaN
3   1.577894e+12         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1     NaN         NaN          NaN            NaN      NaN
4   1.577927e+12         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222          0.0        30000.0  PENDING

In [429]:
final_merged_df = pd.merge(account_card_join_df,df_saving_accounts,on=["savings_account_id","status"],how="outer")
final_merged_df["timestamp"] = final_merged_df["timestamp_y"].combine_first(final_merged_df["timestamp_x"])
cols = ["timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id","card_number","monthly_limit","status","credit_used","balance","interest_rate_percent"]
final_merged_df = final_merged_df.sort_values(["timestamp"])
print(f"\nFinal Table: \n\n",final_merged_df[cols].to_string())


Final Table: 

        timestamp account_id     name   address phone_number                    email savings_account_id card_id card_number  monthly_limit   status  credit_used  balance  interest_rate_percent
0   1.577864e+12         a1  Anthony  New York     12345678     anthony@somebank.com                NaN     NaN         NaN            NaN      NaN          NaN      NaN                    NaN
1   1.577866e+12         a1  Anthony  New York     87654321     anthony@somebank.com                NaN     NaN         NaN            NaN      NaN          NaN      NaN                    NaN
23  1.577891e+12         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0   ACTIVE      19000.0      0.0                    1.5
39  1.577891e+12         a1  Anthony   Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0   ACTIVE          0.0      0.0                    1.5
47  1.577891e+12  

In [424]:
no3 = "No.3"
no3 = no3.center(100,"=")
print(f"\n",no3)


 ================================================No.3================================================


In [430]:
final_merged_df = final_merged_df[cols].dropna()
c1_df = final_merged_df.where(final_merged_df["card_id"] == "c1").dropna()
c1_df["balance_changed"] = c1_df.groupby("card_id")["balance"].transform(lambda x: x.shift() !=x)
c1_df["credit_changed"] = c1_df.groupby("card_id")["credit_used"].transform(lambda x: x.shift() !=x)
c1_df["is_transaction"] = c1_df["balance_changed"] | c1_df["credit_changed"]

total_transaction_c1 = c1_df["is_transaction"].sum()

cols = ["timestamp","account_id","name","address","phone_number","email","savings_account_id","card_id","card_number","monthly_limit","status","credit_used","balance"]

print(f"\nHistorical Table of Anthoy with card_id c1: \n\n",c1_df[cols].to_string())
print(f"Total transaction for card c1: ",total_transaction_c1)


Historical Table of Anthoy with card_id c1: 

        timestamp account_id     name  address phone_number                    email savings_account_id card_id card_number  monthly_limit  status  credit_used  balance
23  1.577891e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0  ACTIVE      19000.0      0.0
7   1.577891e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0  ACTIVE          0.0      0.0
15  1.577891e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0  ACTIVE      12000.0      0.0
31  1.577891e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1    11112222        30000.0  ACTIVE          0.0      0.0
32  1.577956e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c1  

In [431]:
c2_df = final_merged_df.where(final_merged_df["card_id"] == "c2").dropna()
c2_df["balance_changed"] = c2_df.groupby("card_id")["balance"].transform(lambda x: x.shift() !=x)
c2_df["credit_changed"] = c2_df.groupby("card_id")["credit_used"].transform(lambda x: x.shift() !=x)

c2_df["is_transaction"] = c2_df["balance_changed"] | c2_df["credit_changed"]
total_transaction_c2 = c2_df["is_transaction"].sum()

print(f"\nHistorical Table of Anthoy with card_id c2: \n\n",c2_df[cols].to_string())
print(f"Total transaction for card c2: ",total_transaction_c2)


Historical Table of Anthoy with card_id c2: 

        timestamp account_id     name  address phone_number                    email savings_account_id card_id card_number  monthly_limit  status  credit_used  balance
39  1.577891e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0  ACTIVE          0.0      0.0
47  1.577891e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0  ACTIVE      37000.0      0.0
40  1.577956e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0  ACTIVE          0.0  15000.0
48  1.577956e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2    12123434        70000.0  ACTIVE      37000.0  15000.0
49  1.578159e+12         a1  Anthony  Jakarta     87654321  anthony@anotherbank.com                sa1      c2  